In [0]:
-- drop table if exists dev.bi.s12n_partner_cluster_ranks_mgtv;
-- create table dev.bi.s12n_partner_cluster_ranks_mgtv 
-- as 
-- select * from dev.general.s12n_partner_cluster_ranks;

-- select base_phoenix_specialization_id,count(*) as cnt from dev.bi.s12n_estimated_b2c_revenue_aggregated_adjusted_mgtv group by 1 order by 2 desc limit 100
-- where base_phoenix_specialization_id = '--I-GVG8TjSxWxqmqcbomg'

-- select max(date_month) from dev.bi.s12n_estimated_b2c_revenue_aggregated_mgtv

# Adjusting Revenue for content with less than 12 months

In [0]:
%sql
create or replace view dev.bi.rolling_finance_new_content_curves_mgtv
as
select month,specialization,
SUM(specialization) OVER (ORDER BY month ROWS BETWEEN UNBOUNDED PRECEDING and CURRENT ROW) AS rolling_s12n_multiple,
SUM(course) OVER (ORDER BY month ROWS BETWEEN UNBOUNDED PRECEDING and CURRENT ROW) AS rolling_course_multiple
from coursera_warehouse.edw_bi.finance_new_content_curves
where month <= 12;

select * from dev.bi.rolling_finance_new_content_curves_mgtv

month,specialization,rolling_s12n_multiple,rolling_course_multiple
1,0.0143,0.0143,0.0878
2,0.0529,0.06720000000000001,0.2187
3,0.0735,0.1407,0.3169
4,0.0842,0.2249,0.41000000000000003
5,0.0926,0.3175,0.49510000000000004
6,0.0987,0.4162,0.5771000000000001
7,0.0989,0.5151,0.6537000000000001
8,0.0988,0.6139,0.7222000000000001
9,0.0971,0.711,0.7938000000000001
10,0.0959,0.8069,0.8631000000000001


In [0]:
create or replace view dev.bi.s12n_estimated_b2c_revenue_aggregated_adjusted_mgtv
as
with revenue_data as (select *,
                      DATEDIFF(MONTH , launch_month, (select max(date_month) from dev.bi.s12n_estimated_b2c_revenue_aggregated_mgtv)) + 1 as mtd
                      from dev.bi.s12n_estimated_b2c_revenue_aggregated_mgtv
                      where launch_month = date_month)
select 
a.date_month,
a.launch_month,
a.base_phoenix_specialization_id,
-- a.rolling_n12m_b2c_revenue,
-- f.rolling_s12n_multiple,
case when a.mtd >12 then a.rolling_n12m_b2c_revenue else a.rolling_n12m_b2c_revenue/f.rolling_s12n_multiple end as y1_b2c_revenue
from revenue_data a
left join dev.bi.rolling_finance_new_content_curves f on a.mtd = f.month;

select * from dev.bi.s12n_estimated_b2c_revenue_aggregated_adjusted_mgtv 
where base_phoenix_specialization_id = '--I-GVG8TjSxWxqmqcbomg'

date_month,launch_month,base_phoenix_specialization_id,y1_b2c_revenue
2024-10-01,2024-10-01,--I-GVG8TjSxWxqmqcbomg,345593.9793866612


In [0]:
create or replace view dev.bi.course_estimated_b2c_revenue_aggregated_adjusted_mgtv
as
with revenue_data as (select *,
                      DATEDIFF(MONTH , launch_month, (select max(date_month) from dev.bi.course_estimated_b2c_revenue_aggregated_mgtv)) + 1 as mtd
                      from dev.bi.course_estimated_b2c_revenue_aggregated_mgtv
                      where launch_month = date_month)
select 
a.date_month,
a.launch_month,
a.course_id,
case when a.mtd >12 then a.rolling_n12m_b2c_revenue else a.rolling_n12m_b2c_revenue/f.rolling_course_multiple end as y1_b2c_revenue
from revenue_data a
left join dev.bi.rolling_finance_new_content_curves f on a.mtd = f.month;

select * from dev.bi.course_estimated_b2c_revenue_aggregated_adjusted_mgtv 
where course_id = '-DO2L_sHEeaj0BKqLXpeVg'

date_month,launch_month,course_id,y1_b2c_revenue
2017-07-01,2017-07-01,-DO2L_sHEeaj0BKqLXpeVg,0.0


# Adding Partner ID and Partner Cluster

In [0]:
drop table if exists dev.bi.s12n_prelaunch_v0_mgtv;
create table dev.bi.s12n_prelaunch_v0_mgtv as
SELECT a.*,
DATEDIFF(MONTH , a.launch_month, (select max(date_month) from dev.bi.s12n_estimated_b2c_revenue_aggregated_mgtv)) + 1 as mtd,
quarter(a.launch_month) as launch_qtr,
year(a.launch_month) as launch_year,
month(a.launch_month) as launch_month_nbr,
e.difficulty_level,
e.last_update_diff_months,
b.partner_id,
d.partner_slug,
c.cluster_rank as partner_revenue_rank,
case when partner_slug = 'google' then 1 else 0 end as gwg_flag,
case when partner_slug = 'meta' then 1 else 0 end as meta_flag,
case when partner_slug = 'microsoft' then 1 else 0 end as ms_flag,
case when partner_slug in ('ibm','ibm-skills-network') then 1 else 0 end as ibm_flag,
case when partner_slug in ('aws','amazon') then 1 else 0 end as aws_flag,
case when phoenix_specialization_primary_domain_id in ('computer-science') then 1 else 0 end as cs_flag,
case when phoenix_specialization_primary_domain_id in ('business') then 1 else 0 end as bs_flag,
case when phoenix_specialization_primary_domain_id in ('data-science') then 1 else 0 end as ds_flag,
case when phoenix_specialization_primary_domain_id in ('information-technology') then 1 else 0 end as it_flag,
case when phoenix_specialization_slug ilike ('%-ai-%') or phoenix_specialization_slug ilike ('%-ai') then 1 else 0 end as ai_flag
from dev.bi.s12n_item_info_pivot_mgtv a
left join dev.bi.content_dim_kpi b on a.base_phoenix_specialization_id = b.underlying_product_item_id
left join dev.bi.s12n_partner_cluster_ranks_mgtv c on b.partner_id = c.partner_id
left join prod.gold.partner_vw d on b.partner_id = d.partner_id
left join dev.bi.content_info_view_mgtv e on a.base_phoenix_specialization_id = e.base_phoenix_specialization_id;

num_affected_rows,num_inserted_rows


# Joining all the tables

In [0]:
%sql
create or replace view dev.bi.s12n_prelaunch_v1_mgtv as
SELECT a.*, 
cast(b.pd_sum_rolling_n12m_rev_rec as float) as pd_sum_rolling_n12m_rev_rec,
cast(c.sd_sum_rolling_n12m_rev_rec as float) as sd_sum_rolling_n12m_rev_rec,
cast(p.pr_sum_rolling_n12m_rev_rec as float) as pr_avg_rolling_n12m_rev_rec, 
cast(d.pcr_avg_rolling_n12m_rev_rec as float) as pcr_avg_rolling_n12m_rev_rec, 
cast(i.sd_pcr_sum_rolling_n12m_rev_rec as float) as sd_pcr_sum_rolling_n12m_rev_rec, 
-- case when all_subdomain_enrollments_l12m > 0 then CAST(paid_subdomain_enrollments_l12m*1.0/all_subdomain_enrollments_l12m AS FLOAT) else 0.0 end as per_paid_enrl,
sd_avg_rolling_n12m_impressions,
sd_avg_rolling_n12m_views,
sd_avg_rolling_n12m_visits,
sd_avg_rolling_n12m_users,
sd_avg_rolling_enrollments,
sd_avg_rolling_paid_enrollments,
case when sd_avg_rolling_enrollments > 0 then CAST(sd_avg_rolling_paid_enrollments*1.0/sd_avg_rolling_enrollments AS FLOAT) else 0.0 end as sd_per_paid_enrl,
j.y1_b2c_revenue,
k.rolling_n12m_marketing_spend as y1_marketing_spend
FROM dev.bi.s12n_prelaunch_v0_mgtv a
left join dev.bi.content_s12n_prdomain_rolling_rev_rec_last12M_mgtv b on a.phoenix_specialization_primary_domain_id = b.content_domain and a.launch_month = b.dt_month 
left join dev.bi.content_s12n_subdomain_rolling_rev_rec_last12M_mgtv c on a.phoenix_specialization_primary_subdomain_id = c.content_sub_domain and a.launch_month = c.dt_month 
left join dev.bi.content_s12n_partner_cluster_rolling_rev_rec_last12M_mgtv d on a.partner_revenue_rank = d.cluster_rank and a.launch_month = d.dt_month
left join dev.bi.content_s12n_partner_rolling_rev_rec_last12M_mgtv p on a.partner_id = p.partner_id and a.launch_month = p.dt_month
left join dev.bi.content_info_view_mgtv e on a.base_phoenix_specialization_id = e.base_phoenix_specialization_id
left join dev.bi.content_visitors_monthly_history_view_mgtv f 
on a.phoenix_specialization_primary_subdomain_id = f.content_sub_domain and a.partner_revenue_rank = f.cluster_rank and a.launch_month= f.dt_month
left join dev.bi.content_impressions_monthly_history_view_mgtv g 
on a.phoenix_specialization_primary_subdomain_id = g.content_sub_domain and a.partner_revenue_rank = g.cluster_rank and a.launch_month= g.dt_month
left join dev.bi.content_s12n_enollments_monthly_history_view_mgtv h 
on a.phoenix_specialization_primary_subdomain_id = h.content_sub_domain and a.partner_revenue_rank = h.cluster_rank and a.launch_month= h.dt_month
left join dev.bi.content_s12n_subdomain_pcr_rolling_rev_rec_last12M_mgtv i on a.partner_revenue_rank = i.cluster_rank and a.phoenix_specialization_primary_subdomain_id = i.content_sub_domain and a.launch_month = i.dt_month
left join dev.bi.s12n_estimated_b2c_revenue_aggregated_adjusted_mgtv j on a.base_phoenix_specialization_id = j.base_phoenix_specialization_id
left join dev.bi.s12n_marketing_spend_aggregated_mgtv  k on a.base_phoenix_specialization_id = k.base_phoenix_specialization_id and a.launch_month = k.date_month
where year(a.launch_month) <= 2025
  and year(a.launch_month) >= 2020
;


In [0]:
drop table if exists dev.bi.s12n_prelaunch_data_model_input_mgtf;
create table dev.bi.s12n_prelaunch_data_model_input_mgtf as
SELECT * from dev.bi.s12n_prelaunch_v1_mgtv;

-- select * from dev.bi.s12n_prelaunch_data_model_input_mgtf 
-- where base_phoenix_specialization_id = 'dVxvpVBzSI2qpyGsH77oMQ'

num_affected_rows,num_inserted_rows


base_phoenix_specialization_id,phoenix_specialization_slug,phoenix_specialization_primary_domain_id,phoenix_specialization_primary_subdomain_id,is_specialization,is_professional_certificate,is_gateway_certificate,launch_month,number_of_courses,total_items,discussionPrompt,lecture,ltiTool,peerReview,placeholder,programmingAssignment,quiz,reading,staffGraded,ungradedLab,widget,mtd,launch_qtr,launch_year,launch_month_nbr,difficulty_level,last_update_diff_months,partner_id,partner_slug,partner_revenue_rank,gwg_flag,meta_flag,ms_flag,ibm_flag,aws_flag,cs_flag,bs_flag,ds_flag,it_flag,ai_flag,pd_sum_rolling_n12m_rev_rec,sd_sum_rolling_n12m_rev_rec,pr_avg_rolling_n12m_rev_rec,pcr_avg_rolling_n12m_rev_rec,sd_pcr_sum_rolling_n12m_rev_rec,sd_avg_rolling_n12m_impressions,sd_avg_rolling_n12m_views,sd_avg_rolling_n12m_visits,sd_avg_rolling_n12m_users,sd_avg_rolling_enrollments,sd_avg_rolling_paid_enrollments,sd_per_paid_enrl,y1_b2c_revenue,y1_marketing_spend
dVxvpVBzSI2qpyGsH77oMQ,ibm-ai-product-manager,information-technology,support-and-operations,false,true,true,2024-04-01,10,632,40,250,17,4,null,null,4,85,79,null,153,11,2,2024,4,0,14,348,ibm-skills-network,3.0,0,0,0,1,0,0,0,0,1,1,3608814.2,1758068.9,3091202.0,308118.4,74719.664,1247667,456209,127606,215808,14991,3045,0.2031218707561493,1120963.2454204897,1.4184203936666999E7


In [0]:
select * from dev.bi.s12n_prelaunch_data_model_input_view_wmvt_mgtf  where base_phoenix_specialization_id = 'dVxvpVBzSI2qpyGsH77oMQ'

-- select year(launch_month) as yy,count(distinct base_phoenix_specialization_id) as cnt,sum(y1_b2c_revenue) as revenue
-- --  from dev.bi.s12n_prelaunch_data_model_input_mgtf 
--  from  dev.bi.s12n_estimated_b2c_revenue_aggregated_adjusted_mgtv
--  group by 1 order by 1

base_phoenix_specialization_id,phoenix_specialization_slug,phoenix_specialization_primary_domain_id,phoenix_specialization_primary_subdomain_id,is_specialization,is_professional_certificate,is_gateway_certificate,difficulty_level,launch_month,mtd,launch_month_nbr,launch_qtr,launch_year,last_update_diff_months,number_of_courses,total_items,partner_id,partner_slug,partner_revenue_rank,y1_marketing_spend,y1_b2c_revenue,discussionPrompt,lecture,ltiTool,peerReview,placeholder,programmingAssignment,quiz,reading,staffGraded,ungradedLab,widget,gwg_flag,meta_flag,ms_flag,ibm_flag,aws_flag,cs_flag,bs_flag,ds_flag,it_flag,ai_flag,pcr_avg_rolling_n12m_rev_rec,pr_avg_rolling_n12m_rev_rec,sd_pcr_sum_rolling_n12m_rev_rec,sd_avg_rolling_enrollments,sd_avg_rolling_n12m_impressions,sd_avg_rolling_n12m_users,sd_avg_rolling_n12m_views,sd_avg_rolling_n12m_visits,sd_avg_rolling_paid_enrollments,sd_sum_rolling_n12m_rev_rec,sd_per_paid_enrl,sd_per_enrl_rate
dVxvpVBzSI2qpyGsH77oMQ,ibm-ai-product-manager,information-technology,support-and-operations,false,true,true,0,2024-04-01,11,4,2,2024,14,10,632,348,ibm-skills-network,3.0,1.4184203936666999E7,1120963.2454204897,40,250,17,4,null,null,4,85,79,null,153,0,0,0,1,0,0,0,0,1,1,308118.40625,3091202.0,74719.6640625,14991.0,1247667.0,215808.0,456209.0,127606.0,3045.0,1758068.875,0.2031218707561493,0.06946452707052231
